# MULTIPLICACION DE MATRICES CON MPI

Laura Sophia González Montoya - HPC UTP 

## Especificaciones de maquina
Encontradas en los archivos pc_info.txt y gpu_info.txt dentro del repositorio

# Desarrollo

La manera en la que se abordo el problema es la siguiente

## El cluster

Se contaba con un cluster dentro de la plataforma de AWS, el cual fue montado en dias anteriores, el cual seria nuestro escenario para la implementacion del mpi y el paralelismo de los procesos dentro de una multiplicacion de matrices

Las especificaciones de las maquinas utilizadas estan en los archivos de texto adjuntados dentro de este trabajo y mencionados anteriormente

## Servidor de archivos NFS

### ¿Que es NFS?

"NFS (sistema de archivos de red: «Network File System») es un protocolo que permite acceso remoto a un sistema de archivos a través de la red. Todos los sistemas Unix pueden trabajar con este protocolo; cuando se involucran sistemas Windows, debe utilizar Samba en su lugar." (https://debian-handbook.info/browse/es-ES/stable/sect.nfs-file-server.html)

### ¿Cómo se utilizó?

El principal objetivo de este protocolo era el de hacer un depliegue de las matrices dentro de un archivo .csv desde el head a todas las maquinas, tanto de las matrices de entrada como la matriz resultante  

Lo que se realizo fue un despliegue de la carpeta principal llamada "HPC-mpi" ubicada en el head a traves del protocolo, esta carpeta contenia codigo, ejecutables y por supuesto las matrices

### ¿Como se generaron las matrices de entrada?

Se creó un codigo llamado "generador.cpp" el cual implementando la libreria "#include <fstream>" permite generar matrices de diferentes tamaños y guardarlas en un archivo .csv 

## Y entonces... ¿Como se abordo el problema?

Bueno, como ya se mencionó anteriormente. Se hizo despliegue de las matrices gracias al protocolo NFS, pero, ¿Con que fin? se estarán preguntando

Sin este sistema lo que se hubiera tenido que hacer era mandar cada fila de la matriz por MPI 
y volver a traer toda la fila resultante, un proceso un tanto pesado sobre todo cuando nuestro fin es el de la optimización.

Gracias despliegue solo tenemoss que mandar la posicion de la fila dentro de la matriz a la cual queremos acceder y buscarla dentro del archivo .csv, evitandonos todo ese paso de mensajes tan costoso

### ¿Como se realizo la multiplicación?

Se le realizo la transpuesta a una de las matrices, con el fin de no multiplicar filas x columnas, sino filas x filas

la idea es mandar la primera fila de la matrizA por ejemplo y que esta multiplicandose con las filas de la MatrizB sacaran como resultado la primera fila de la matriz resultante. De esta manera a cada nodo se le asigna el trabajo de sacar una fila resultante 

Nota: El nodo head solo se le asigna como tarea sacar la primera fila de la matriz, las demas se reparten entre los nodos trabajadores

![idea.jpeg](idea.jpeg)

## Que sucedio a la hora de la implementacion

En primer lugar, no se logró guardar la matriz resultante dentro de un archivo .csv puesto que se sobreescribia cada vez que un nodo worker realizaba una multiplizacion y la almacenaba. Pero para probar que el codigo corria correctamenta se mostro por consola el resultado que sacaba cada nodo y se verificaba que estuvieran correctos. Se puede ver mas a detalle en la siguiente imagen.

![mm.png](mm.png)

Una vez corroborado de que el código corriera de manera correcta, se empezo a realizar la multiplicacion de distintos tamaños y sus tiempos de ejecucion 

Se inicio a tomar los tiempos con un tamaño de matriz de 100 lo cual produjo un tiempo promedio de 8.58857, lo cual es muy elevado considerando el tamaño de la matriz,Se continuo con una matriz de 250 con un tiempo de 250,54.8644.
De aqui en adelante salía un error por tiempo de espera excedido o algo por el estilo


El siguiente error me salío al correr la matriz de 500 

![error.png](error.png)

De aqui en adelante sucedia lo mismo para matrices de tamaño superior, lo cual impidio el analisis correcto de mpi y su comparacion con otros metodos de paralización